# Wind Impact on Blending Agent Performance

How robust is the RL architecture trained on to blend other RL policies to all wind?

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from utilities import get_agent
from environments import OctorotorEnvSelector
from systems.pid_agent import PIDAgent
from multirotor.helpers import DataLog

In [5]:
study, best_agent, best_params = get_agent('blending@softmax@scaled', filepath='BlendingEnv')

best_params['steps_u']=50
best_params['scaling_factor']=5
best_params['bounding_rect_length']=200
best_params['window_distance']=10

[I 2023-09-13 21:30:48,627] Using an existing study with name 'blending@softmax@scaled' instead of creating a new one.


In [6]:
env_selector = OctorotorEnvSelector()
square_wpts = np.array([[100,0,0], [100,100,0], [0,100,0], [0,0,0]])

In [7]:
def get_tte(initial_pos: tuple, waypoints: np.ndarray, x: np.ndarray, y:np.ndarray, z:np.ndarray) -> np.ndarray:
        """
        Calculates the trajectory tracking error. 
        The distance between the current point and the vector between previous and next wp. Uses ||v1 x v2|| / ||v1||.

        Parameters
        ----------
        initial_pos : tuple  
            the initial position of the UAV.
        waypoints : np.ndarray 
            the reference positions at each point in time.
        x : np.ndarray 
            the x positions of the UAV.
        y : np.ndarray 
            the y positions of the UAV.
        z : np.ndarray
            the z positions of the UAV.

        Returns
        -------
        np.ndarray 
            the trajectory tracking error at each point in time.
        """
        ttes = []
        prev = initial_pos
        for i, waypoint in enumerate(waypoints):
            if i > 0 and not np.array_equal(waypoints[i-1], waypoints[i]):
                prev = waypoints[i-1]

            v1 = waypoint - prev
            v2 = np.array([x[i],y[i],z[i]]) - prev
            tte = np.linalg.norm(np.cross(v1, v2)) / (np.linalg.norm(v1) + 1e-6)
            ttes.append(tte)
                
        return np.array(ttes)

In [8]:
def toc(tte: np.ndarray):
    corridor = 5
    return len(tte[tte > corridor]) / 2 # every half of a second we record data

In [9]:
def completed_mission(waypoints: np.ndarray, x: np.ndarray, y: np.ndarray, z: np.ndarray, radius: float = 0.65):
        for waypoint in waypoints:
            reached_waypoint = False

            for position in zip(x,y,z):
                dist = np.linalg.norm(waypoint - position)

                if dist <= radius:
                    reached_waypoint = True
                    break

            if not reached_waypoint:
                return False
            
        return True

In [10]:
def run_trajectory(env_selector, wind_ranges: np.ndarray, agent, params):
    env = env_selector.get_env("blending", params, wind_ranges, square_wpts)
    done = False
    state = np.array(env.reset(), dtype=np.float32)
    log = DataLog(env.base_env.vehicle, env.base_env.ctrl,
                      other_vars=('reward',))
    while not done:
        action = agent.predict(state, deterministic=True)[0]
        state, reward, done, info = env.step(action)
        state = np.array(state, dtype=np.float32)
        log.log(reward=reward)

    log.done_logging()
    return log, info

In [11]:
wind_range_dict = {
    'zero': [(0,0), (0,0), (0,0)],
    'n5': [(0,0), (5,5), (0,0)],
    'n7': [(0,0), (7,7), (0,0)],
    'n10': [(0,0), (10,10), (0,0)],
    's5': [(0,0), (-5,-5), (0,0)],
    's7': [(0,0), (-7,-7), (0,0)],
    's10': [(0,0), (-10,-10), (0,0)],
    'e5': [(5,5), (0,0), (0,0)],
    'e7': [(7,7), (0,0), (0,0)],
    'e10': [(10,10), (0,0), (0,0)],
    'w5': [(-5,-5), (0,0), (0,0)],
    'w7': [(-7,-7), (0,0), (0,0)],
    'w10': [(-10,-10), (0,0), (0,0)],
    'nw5': [(-3.53553391,-3.53553391), (3.53553391,3.53553391), (0,0)],
    'nw7': [(-4.94974747,-4.94974747), (4.94974747,4.94974747), (0,0)],
    'nw10': [(-7.07106781,-7.07106781), (7.07106781,7.07106781), (0,0)],
    'sw5': [(-3.53553391,-3.53553391), (-3.53553391,-3.53553391), (0,0)],
    'sw7': [(-4.94974747,-4.94974747), (-4.94974747,-4.94974747), (0,0)],
    'sw10': [(-7.07106781,-7.07106781), (-7.07106781,-7.07106781), (0,0)],
    'ne5': [(3.53553391,3.53553391), (3.53553391,3.53553391), (0,0)],
    'ne7': [(4.94974747,4.94974747), (4.94974747,4.94974747), (0,0)],
    'ne10': [(7.07106781,7.07106781), (7.07106781,7.07106781), (0,0)],
    'se5': [(3.53553391,3.53553391), (-3.53553391,-3.53553391), (0,0)],
    'se7': [(4.94974747,4.94974747), (-4.94974747,-4.94974747), (0,0)],
    'se10': [(7.07106781,7.07106781), (-7.07106781,-7.07106781), (0,0)],
}

In [12]:
wind_results = pd.DataFrame(columns=['Wind', 'Total TTE', 'Mean TTE', 'Completed Mission', 'Reward', 'Time Outside Corridor'])

In [13]:
def run_wind_sweep(results, wind_dict, agent, params):
    for wind in tqdm(wind_dict.keys()):
        log, info = run_trajectory(env_selector, wind_dict[wind], agent, params)
        traj_err = get_tte(np.array([0,0,0]), log.target.position, log.x, log.y, log.z)
        new_result = {
            'Wind': wind,
            'Mean TTE': np.mean(traj_err),
            'Total TTE': np.sum(traj_err),
            'Completed Mission': completed_mission(square_wpts, log.x, log.y, log.z, radius=5),
            'Reward': np.sum(log.reward),
            'Time Outside Corridor': toc(traj_err)
        }
        results = pd.concat([results, pd.DataFrame([new_result])], ignore_index=True)

    return results

In [14]:
wind_results = run_wind_sweep(wind_results, wind_range_dict, best_agent, best_params)

  0%|                                                                                | 0/25 [00:00<?, ?it/s][I 2023-09-13 21:30:50,609] Using an existing study with name 'highwind' instead of creating a new one.
[I 2023-09-13 21:30:50,706] Using an existing study with name 'midwind' instead of creating a new one.
  4%|██▉                                                                     | 1/25 [00:08<03:32,  8.87s/it][I 2023-09-13 21:30:59,470] Using an existing study with name 'highwind' instead of creating a new one.
[I 2023-09-13 21:30:59,542] Using an existing study with name 'midwind' instead of creating a new one.
  8%|█████▊                                                                  | 2/25 [00:14<02:37,  6.83s/it][I 2023-09-13 21:31:04,874] Using an existing study with name 'highwind' instead of creating a new one.
[I 2023-09-13 21:31:04,944] Using an existing study with name 'midwind' instead of creating a new one.
 12%|████████▋                                         

In [15]:
wind_results.to_csv('./data/blending_wind_scaled.csv')

In [16]:
wind_results = pd.read_csv('./data/blending_wind_scaled.csv') # or read cardinal_wind.csv for the Cardinal High agent

In [17]:
wind_results

,Unnamed: 0,Wind,Total TTE,Mean TTE,Completed Mission,Reward,Time Outside Corridor
0,0,zero,130.535094,0.630604,True,8641.6300,0.0
1,1,n5,151.100457,0.729954,True,8440.4760,0.0
2,2,n7,238.829892,1.137285,True,7588.0690,0.0
3,3,n10,2325.804942,9.454492,True,-13188.3850,46.5
4,4,s5,210.338585,1.006405,True,7866.0625,0.0
5,5,s7,305.246139,1.439840,True,6931.8470,0.0
6,6,s10,770.344387,3.196450,True,2376.1775,19.0
7,7,e5,144.539693,0.698259,True,8505.6430,0.0
8,8,e7,222.113510,1.057683,True,7752.1104,0.0
9,9,e10,2124.915283,9.042193,True,-11233.8180,32.0


In [18]:
print("Mean reward for 5 m/s: ", wind_results[wind_results['Wind'].str.contains("5")]['Reward'].mean())
print("Std reward for 5 m/s: ",wind_results[wind_results['Wind'].str.contains("5")]['Reward'].std())
print("Mean reward for 7 m/s: ", wind_results[wind_results['Wind'].str.contains("7")]['Reward'].mean())
print("Std reward for 7 m/s: ",wind_results[wind_results['Wind'].str.contains("7")]['Reward'].std())
print("Mean reward for 10 m/s: ", wind_results[wind_results['Wind'].str.contains("10")]['Reward'].mean())
print("Std reward for 10 m/s: ",wind_results[wind_results['Wind'].str.contains("10")]['Reward'].std())

Mean reward for 5 m/s:  8285.1128875
Std reward for 5 m/s:  366.130854458486
Mean reward for 7 m/s:  7477.2097125
Std reward for 7 m/s:  539.2844442846609
Mean reward for 10 m/s:  -404.5792499999998
Std reward for 10 m/s:  7956.336588320598
